We are going to build a Simple Agent System for generating cold sales outreach emails:
1. Agent Workflow
2. Use of tools to call functions
3. Agent collaboration via tools and Hanfoffs

In [25]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [17]:
load_dotenv(override=True)

True

## **Step 1. Agent Workflow**

In [18]:
instruction1 = """
    You are a sales agent working for ComplAI, 
    a company that provides SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
    You write professional, serious cold emails.
"""

instruction2 = """
    You are a humorous, engaging sales agent working for ComplAI,
    a company that provides SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
    You write witty, engaging cold emails that are likely to get a response.
"""

instruction3 = """
    You are a busy sales agent working for ComplAI,
    a company that provides SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
    You write concise, to the point cold emails that are likely to get a response.
"""

In [19]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instruction1,
    model = 'gpt-4.1-nano'
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instruction2,
    model='gpt-4.1-nano'
)

sales_agent3 = Agent(
    name = "Busy Sales Agent", 
    instructions=instruction3,
    model = 'gpt-4.1-nano'
)

In [ ]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email.")

# boiler-plate code for printing the text output

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify Your SOC2 Compliance Process with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. As organizations like yours strive to meet evolving security standards, maintaining SOC2 compliance remains a critical yet complex challenge.

At ComplAI, we offer an AI-powered SaaS platform designed to streamline your compliance journey—reducing manual effort, identifying gaps proactively, and preparing you thoroughly for audits.

Would you be open to a brief conversation to see how ComplAI can help simplify and accelerate your SOC2 compliance process?

Looking forward to your response.

Best regards,  
[Your Name]  
[Your Position]  
ComplAI  
[Your Contact Information]

In [ ]:
# Printing all cold emails

message = "Write a cold sales email."

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message)
    )


outputs = [result.final_output for result in results]

for output in outputs:
    print(output , "\n\n")

Subject: Simplify Your SOC2 Compliance and Audit Preparation with ComplAI

Dear [Recipient’s Name],

I hope this message finds you well. As your organization continues to grow, maintaining SOC2 compliance and preparing for audits can become increasingly complex and resource-intensive.

At ComplAI, we offer an AI-powered SaaS platform designed to streamline your compliance processes, automate documentation, and ensure readiness for audits—saving you time, reducing risks, and providing peace of mind.

Would you be open to a brief conversation to explore how ComplAI can support your compliance goals?

Looking forward to your response.

Best regards,  
[Your Name]  
[Your Position]  
ComplAI  
[Your Contact Information] 


Subject: Is Your SOC2 Compliance as Smooth as Your Coffee? ☕

Hi [Name],

Imagine tackling SOC2 audits with the ease of brewing your morning coffee—no spilled beans, just perfectly brewed compliance. Sounds dreamy, right? That’s exactly what ComplAI can do for your team.

In [36]:
# Agent to pick best cold sales email

sales_picker = Agent(
    name="Sales Picker", 
    instructions="""
        You have to pick the best cold sales email out of all that are provided.
        Imagine you are a customer and pick the email that you are most likely to respond to.
        Do not give an explanation, just reply only and only with the best cold sales email.
    """,
    model='gpt-4o-mini'
)

In [38]:
# Picking the best cold sales email

message = "Write a cold sales email."

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

    outputs = [result.final_output for result in results]

    emails = "Cold sales emails: \n\n".join(outputs)

    best = await Runner.run(sales_picker, input=emails)

    print(f"Best sales email: \n\n {best.final_output}")

Best sales email: 

 Subject: Is your SOC2 compliance feeling more like a minefield than a roadmap? 🚧📝

Hi [First Name],

Let’s be honest — navigating SOC2 compliance can sometimes feel like trying to assemble furniture without the instructions. You know the pain: endless documentation, bewildering audits, and a looming deadline that keeps you awake at night.

That’s where ComplAI comes in. We’re like the friendly GPS for your compliance journey — powered by AI, tuned for simplicity, and designed to get you across the finish line without losing your mind.

How about a quick chat? I promise it’ll be more fun than reading SP 800-171... and with better results.

Looking forward to helping you turn compliance chaos into a well-oiled machine.

Best,  
[Your Name]  
ComplAI — Your AI Sidekick for SOC2 Success  
[Your Contact Info]


> Check the traces here: 
https://platform.openai.com/traces

## **Steps 2. and 3. Tools and Agent Interactions**

In [39]:
# What does the sales_agent looks like? 

sales_agent1

Agent(name='Professional Sales Agent', handoff_description=None, tools=[], mcp_servers=[], mcp_config={}, instructions='\n    You are a sales agent working for ComplAI, \n    a company that provides SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.\n    You write professional, serious cold emails.\n', prompt=None, handoffs=[], model='gpt-4.1-nano', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [40]:
# Boiler plate code to send emails using SendGrid

@function_tool
def send_email(body: str):
    """Send out an email with the given body to all the prospects."""

    sg = sendgrid.SendGridAPIClient(api_key=os.getenv("SENDGRID_API_KEY"))
    from_email = Email("jai.300012723027@csvtu.ac.in")
    to_email = To("ksharma9719@gmail.com")
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales Email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {'status': 'success'}

> Because we use the @function_tool decorator just above the fucntion declaration, it has automatically been converted to a Tool, as a result, the boiler plate json has been created automatically.

In [ ]:
# have a look at it

send_email  

FunctionTool(name='send_email', description='Send out an email with the given body to all the prospects.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB00EF1A0>, strict_json_schema=True, is_enabled=True)

#### *We can convert Agent into Tools too!*

In [ ]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email.")
tool1   

# sales_agent1 became a FunctionTool Object now

FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB09A1BC0>, strict_json_schema=True, is_enabled=True)

#### *Gathering all the Tools together!*

In [45]:
message = "Write a cold sales email."

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email.")
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write a cold sales email.")
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a cold sales email.")

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB09A1EE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB09A2DE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'stri

### **Time for the Sales Manager -- Our Planning Agent**

In [ ]:
instructions = """
    You are a Sales Manager working for ComplAI. You use tools provided to you to generate cold sales emails.
    You NEVER generate cold sales emails yourself. You ONLY use tools to generate them.
    You try all the three sales_agent tools before choosing the best one.
    You pick the single best email and send it using send_email tool, ONLY the best email and nothing else.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    model = "gpt-4o-mini",
    tools = tools
)

message = "Send a cold sales email to 'Dear CEO'"

with trace("Sales Manager"):
    result = await Runner.run(sales_manager, message)

# EMail sent successfully

In [48]:
print(result)

RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to the CEO. If you need anything else, feel free to ask!
- 9 new item(s)
- 3 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


> *Remember to chech the traces: https://platform.openai.com/traces*

> *Remember to chech the traces: https://platform.openai.com/traces*

---
Handoffs represent a way an agent can delegate to an agent, passing control to it.
Handoffs and Agents-as-tools are similar:

    1. In both cases, an Agent can collaborate with another Agent  
    2. With tools, control passes back  
    3. With handoffs, control passes across  

--- 

In [49]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(
    name="Email subject writer", 
    instructions=subject_instructions, 
    model="gpt-4o-mini"
)

subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [51]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jai.300012723027@csvtu.ac.in")  # Change to your verified sender
    to_email = To("ksharma9719@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [52]:
tools = [subject_tool, html_tool, send_html_email]

In [53]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB0A00D60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB0A00E00>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_html_email', description='Send out an email with the given subject and HTML body to 

#### Let's try handoff!

In [ ]:


instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

> Now we have 3 tools and 1 handoff !

In [55]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB09A1EE0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000029DB09A2DE0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string

> Sales Manager will handoff to emailer agent

In [58]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

In [57]:
print(result)

RunResult:
- Last agent: Agent(name="Email Manager", ...)
- Final output (str):
    The cold sales email has been successfully sent to the CEO. If you need any further assistance or would like to send another email, feel free to ask!
- 16 new item(s)
- 5 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


> *Remember to chech the traces: https://platform.openai.com/traces*

### **Commercial implications**
This is immediately applicable to Sales Automation; but more generally this could be applied to end-to-end automation of any business process through conversations and tools. Think of ways you could apply an Agent solution like this in your day job.